In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("proj2.ipynb")

# Project 2: Query Performance
## Due Date: Friday 10/07, 11:59 PM
## Assignment Details
In this project, we will explore how the database system optimizes query execution and how users can futher tune the performance of their queries.

This project works with the Lahman's Baseball Database, an open source collection of baseball statistics from 1871 to 2020. It contains a variety of data, like batting statistics, team stats, managerial records, Hall of Fame records, and much more.

**Note:** If at any point during the project, the internal state of the database or its tables have been modified in an undesirable way (i.e. a modification not resulting from the instructions of a question), restart your kernel and clear output and simply re-run the notebook as normal. This will shutdown you current connection to the database, which will prevent the issue of multiple connections to the database at any given point, and when re-running the notebook you will create a fresh database based on the provided Postgres dump.

## Logistics & Scoring Breakdown

For Data 101 students, this project is worth 15% of your grade. For Info 258 students, this project is worth 12% of your grade.

- Each coding question has **both public tests and hidden tests**. Roughly 50% of your coding grade will be made up of your score on the public tests released to you, while the remaining 50% will be made up of unreleased hidden tests.
- Public tests for multiple choice questions are for sanity check only (e.g. you are answering in the correct format). Partial credit will be awarded.
- Free-response questions will be manually graded. Please answer thoughtfully in complete sentences, drawing from knowledge in lectures and from your inspection of query plans.

This is an **individual project**. However, you’re welcome to collaborate with any other student in the class as long as it’s within the academic honesty guidelines.

Question | Points
--- | ---
0 | 1
1a	| 1
1bi	| 1
1bii	| 2
1c	| 1
1di	| 2
1dii	| 1
1ei	| 1
1eii	| 1
1eiii	| 1
1eiv	| 2
1ev	| 2
2a	| 1
2b	| 1
2c	| 2
2d	| 2
3a	| 1
3b	| 1
3c |	1
3d	| 3
4a	| 2
4b	| 1
4c	| 1
4di	| 2
4dii |	2
4ei	| 2
4eii | 	1
4eiii |	2
4eiv |	1
4ev |	1
4evi |	2
4evii |	2
5a | 2
5b | 2
5c | 1
5d | 2
6a | 2
6b | 1
6c | 1
6d | 2
6e | 2
7a | 1
7b | 1
7c | 2
8  | 6
**Total** | 72

In [2]:
# Run this cell to set up imports
import numpy as np
import pandas as pd

## Getting Connected
Similar to Project 1, we will be using the `ipython-sql` library to connect this notebook to a PostgreSQL database server on your JupyterHub account. Run the following cell to initiate the connection.

In [3]:
%reload_ext sql
%sql postgresql://jovyan@127.0.0.1:5432/postgres

## Setting up the Database
The following cell will create the `baseball` database (if needed), unzip the Postgres dump of the Lahman's Baseball Database, populate the `baseball` database with the desired tables and data, and finally display all databases associated with the Postgres instance. After running the cell, you should see the `baseball` database in the generated list of databases outputted by `%sql \l`.

**Note:** If you run into the **role does not exist** error, feel free to ignore it. It does not affect data import.

In [4]:
!psql -h localhost -c 'DROP DATABASE IF EXISTS baseball'
!psql -h localhost -c 'CREATE DATABASE baseball'
!gzip -dc data/lahman.pgdump.gz | psql -h localhost -d baseball -f -
!psql -h localhost -c 'SET max_parallel_workers_per_gather = 0;'
%sql \l

DROP DATABASE
CREATE DATABASE
SET
SET
SET
SET
 set_config 
------------
 
(1 row)

SET
SET
SET
CREATE EXTENSION
COMMENT
SET
CREATE TABLE
psql:<stdin>:49: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:80: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:96: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:112: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:129: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:146: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:178: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:210: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:223: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:251: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:267: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:295: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:322: ERROR:  

Name,Owner,Encoding,Collate,Ctype,Access privileges
baseball,jovyan,UTF8,en_US.utf8,en_US.utf8,None
imdb,jovyan,UTF8,en_US.utf8,en_US.utf8,None
jovyan,jovyan,UTF8,en_US.utf8,en_US.utf8,None
postgres,jovyan,UTF8,en_US.utf8,en_US.utf8,None
template0,jovyan,UTF8,en_US.utf8,en_US.utf8,=c/jovyanjovyan=CTc/jovyan
template1,jovyan,UTF8,en_US.utf8,en_US.utf8,=c/jovyanjovyan=CTc/jovyan


Now, run the following cell to connect to the `baseball` database. There should be no errors after running the following cell.

In [5]:
%sql postgresql://jovyan@127.0.0.1:5432/baseball

To ensure that the connection to the database has been established, let's try grabbing the first 5 rows from the `halloffame` table.

In [6]:
%%sql
SELECT * FROM halloffame LIMIT 5

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
5 rows affected.


playerid,yearid,votedby,ballots,needed,votes,inducted,category,needed_note
cobbty01,1936,BBWAA,226,170,222,Y,Player,None
ruthba01,1936,BBWAA,226,170,215,Y,Player,None
wagneho01,1936,BBWAA,226,170,215,Y,Player,None
mathech01,1936,BBWAA,226,170,205,Y,Player,None
johnswa01,1936,BBWAA,226,170,189,Y,Player,None


Run the following cell for grading purposes.

In [7]:
!mkdir -p results

## Table Descriptions
In its entirety the Lahman's Baseball Database contains 27 tables containing a variety of statistics for players, teams, games, schools, etc. For simplicity, this project will focus on a subset of the tables:
* `appearances`: details on the positions each player appeared at
* `batting`: batting statistics for each player
* `collegeplaying`: list of players and the colleges they attended
* `halloffame`: Hall of Fame voting data
* `people`: player information (name, date of birth, and biographical info)
* `salaries`: player salary data
* `schools`: list of colleges that players attended

As a reminder from Project 1, `%sql \d <table_name>` is helpful for identifying the columns in a table.

## Question 0: PostgreSQL Explain Analyze
**Please read through this section carefully, as a vast majority of the project will require you to inspect query plans via interpreting the output of the explain analyze command.**

To inspect the query plan for a given query, create a variable storing the query as a string and invoke a `psql` shell command to `explain analyze` the query: 

`your_variable = "__REPLACE_ME_WITH_QUERY__"`

`!psql -h localhost -d baseball -c "explain analyze $your_variable"`

Take a look at the following sample query plan.

![title](data/sample_query.png)

It is highly recommended to read through [this article](https://www.cybertec-postgresql.com/en/how-to-interpret-postgresql-explain-analyze-output/) to see how you can interpret the output above. Everything before "Tools to interpret Explain Analyze output" is useful.

Here are some key things to note:
- There are two cost values: the first is the **startup cost** (cost to return the first row) and the second is the **total cost** (cost to return all rows).
- The unit for the estimated query cost is arbitrary (1 is the cost for reading an 8kB page during a sequential scan)
- When we ask you to identify the query cost, we are looking for the **total cost**
- When we ask you to identify the query time, we are looking for the **execution time** (in ms)
- Feel free to round the query cost / time to the nearest integer, but we'll accept anything more exact
- We recognzie that the execution time may vary between different cell executions, so the autograder will tolerate a reasonable range

Now, inspect the query plan above, and record the **cost** and **time** for the sample query.

<!--
BEGIN QUESTION
name: q0
points: 1
-->

In [8]:
sample_query_cost = 1193.88
sample_query_timing = 185.896

In [9]:
grader.check("q0")

q0 results: All test cases passed!

## Question 1: Queries and Views
### Question 1a:
Write a query that finds `namefirst`, `namelast`, `playerid` and `yearid` of all people who were successfully inducted into the Hall of Fame. **Note**: Your query should **NOT** use any sub-queries.

In [10]:
%sql SELECT * FROM halloffame LIMIT 5

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
5 rows affected.


playerid,yearid,votedby,ballots,needed,votes,inducted,category,needed_note
cobbty01,1936,BBWAA,226,170,222,Y,Player,None
ruthba01,1936,BBWAA,226,170,215,Y,Player,None
wagneho01,1936,BBWAA,226,170,215,Y,Player,None
mathech01,1936,BBWAA,226,170,205,Y,Player,None
johnswa01,1936,BBWAA,226,170,189,Y,Player,None


In [11]:
%sql SELECT * FROM people LIMIT 5

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
5 rows affected.


playerid,birthyear,birthmonth,birthday,birthcountry,birthstate,birthcity,deathyear,deathmonth,deathday,deathcountry,deathstate,deathcity,namefirst,namelast,namegiven,weight,height,bats,throws,debut,finalgame,retroid,bbrefid
aardsda01,1981,12,27,USA,CO,Denver,None,None,None,None,None,None,David,Aardsma,David Allan,215,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01
aaronha01,1934,2,5,USA,AL,Mobile,None,None,None,None,None,None,Hank,Aaron,Henry Louis,180,72.0,R,R,1954-04-13,1976-10-03,aaroh101,aaronha01
aaronto01,1939,8,5,USA,AL,Mobile,1984,8,16,USA,GA,Atlanta,Tommie,Aaron,Tommie Lee,190,75.0,R,R,1962-04-10,1971-09-26,aarot101,aaronto01
aasedo01,1954,9,8,USA,CA,Orange,None,None,None,None,None,None,Don,Aase,Donald William,190,75.0,R,R,1977-07-26,1990-10-03,aased001,aasedo01
abadan01,1972,8,25,USA,FL,Palm Beach,None,None,None,None,None,None,Andy,Abad,Fausto Andres,184,73.0,L,L,2001-09-10,2006-04-13,abada001,abadan01


In [12]:
%%sql result_1a <<

SELECT people.namefirst as namefirst, people.namelast as namelast, people.playerid, halloffame.yearid
FROM people 
JOIN halloffame ON people.playerid = halloffame.playerid
WHERE halloffame.inducted = 'Y';

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
323 rows affected.
Returning data to local variable result_1a


In [13]:
result_1a

namefirst,namelast,playerid,yearid
Ty,Cobb,cobbty01,1936
Babe,Ruth,ruthba01,1936
Honus,Wagner,wagneho01,1936
Christy,Mathewson,mathech01,1936
Walter,Johnson,johnswa01,1936
Nap,Lajoie,lajoina01,1937
Tris,Speaker,speaktr01,1937
Cy,Young,youngcy01,1937
Morgan,Bulkeley,bulkemo99,1937
Ban,Johnson,johnsba99,1937


In [14]:
# Do not delete/edit this cell
result_1a.DataFrame().sort_values(['playerid', 'yearid']).to_csv('results/result_1a.csv', index=False)

In [15]:
grader.check("q1a")

q1a results: All test cases passed!

### Question 1b:
In this question, we will compare the query you wrote in `Question 1a` against the provided query in `Question 1bi` by inspecting the query plans.

#### Question 1bi: 
Inspect the query plan for `provided_query` and the query you wrote in `Question 1a`. Record the **execution time** and **cost** for each query.

In [16]:
%%sql provided_query <<
SELECT namefirst, namelast, p.playerid, yearid
FROM people AS p, (SELECT * FROM halloffame WHERE inducted = 'Y') AS hof 
WHERE p.playerid = hof.playerid;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
323 rows affected.
Returning data to local variable provided_query


In [17]:
provided_query

namefirst,namelast,playerid,yearid
Ty,Cobb,cobbty01,1936
Babe,Ruth,ruthba01,1936
Honus,Wagner,wagneho01,1936
Christy,Mathewson,mathech01,1936
Walter,Johnson,johnswa01,1936
Nap,Lajoie,lajoina01,1937
Tris,Speaker,speaktr01,1937
Cy,Young,youngcy01,1937
Morgan,Bulkeley,bulkemo99,1937
Ban,Johnson,johnsba99,1937


In [18]:
provided_query_str = "SELECT namefirst, namelast, p.playerid, yearid FROM people AS p, (SELECT * FROM halloffame WHERE inducted = 'Y') AS hof WHERE hof.inducted = 'Y' AND p.playerid = hof.playerid;"
!psql -h localhost -d baseball -c "explain analyze $provided_query_str"

                                                           QUERY PLAN                                                           
--------------------------------------------------------------------------------------------------------------------------------
 Nested Loop  (cost=0.29..262.79 rows=21 width=278) (actual time=0.060..2.632 rows=323 loops=1)
   ->  Seq Scan on halloffame  (cost=0.00..96.39 rows=21 width=42) (actual time=0.026..0.714 rows=323 loops=1)
         Filter: ((inducted)::text = 'Y'::text)
         Rows Removed by Filter: 3868
   ->  Index Scan using master_pkey on people p  (cost=0.29..7.92 rows=1 width=274) (actual time=0.005..0.005 rows=1 loops=323)
         Index Cond: ((playerid)::text = (halloffame.playerid)::text)
 Planning Time: 0.728 ms
 Execution Time: 2.718 ms
(8 rows)



In [19]:
inducted_hof_query_str = "SELECT people.namefirst as namefirst, people.namelast as namelast, people.playerid, halloffame.yearid FROM people JOIN halloffame ON people.playerid = halloffame.playerid WHERE halloffame.inducted = 'Y'"
!psql -h localhost -d baseball -c "explain analyze $inducted_hof_query_str"

                                                          QUERY PLAN                                                          
------------------------------------------------------------------------------------------------------------------------------
 Nested Loop  (cost=0.29..262.79 rows=21 width=278) (actual time=0.056..2.652 rows=323 loops=1)
   ->  Seq Scan on halloffame  (cost=0.00..96.39 rows=21 width=42) (actual time=0.024..0.651 rows=323 loops=1)
         Filter: ((inducted)::text = 'Y'::text)
         Rows Removed by Filter: 3868
   ->  Index Scan using master_pkey on people  (cost=0.29..7.92 rows=1 width=274) (actual time=0.006..0.006 rows=1 loops=323)
         Index Cond: ((playerid)::text = (halloffame.playerid)::text)
 Planning Time: 0.693 ms
 Execution Time: 2.724 ms
(8 rows)



In [20]:
provided_query_cost = 959.06 
provided_query_timing = 26.612
your_query_cost = 959.06
your_query_timing = 18.093

In [21]:
grader.check("q1bi")

q1bi results: All test cases passed!

#### Question 1bii:
Given your findings from inspecting the query plans of the two queries, answer the following question. Assign the variable `q1b_part2` to a list of answer choices which are true statements.

1. Consider the following statements:
    1. Both the queries have the same cost.
    1. The provided query has a faster execution time because it makes use of a subquery.
    1. The query you wrote has a faster execution time because it does not make use a subquery.
    1. The provided query has less cost because it makes use of a subquery.
    1. The query you wrote has less cost because it does not make use a subquery.
    1. The queries have the same output.
    1. The queries do not have the same output.
    
**Note:** Your answer should look like `q1b_part2 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q1bii
points: 2
-->

In [22]:
q1b_part2 = ['A', 'C', 'G' ]

In [23]:
grader.check("q1bii")

q1bii results: All test cases passed!

### Question 1c:
Write a query that creates a view named `inducted_hof_ca` of the people who were successfully inducted into the Hall of Fame and played in college at a school located in California. For each player, return their `namefirst`, `namelast`, `playerid`, `schoolid`, and `yearid` ordered by the `yearid` and then the `playerid`. 

**Note**: For this query, `yearid` refers to player's year of induction into the Hall of Fame.

In [24]:
%sql SELECT * FROM halloffame LIMIT 5

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
5 rows affected.


playerid,yearid,votedby,ballots,needed,votes,inducted,category,needed_note
cobbty01,1936,BBWAA,226,170,222,Y,Player,None
ruthba01,1936,BBWAA,226,170,215,Y,Player,None
wagneho01,1936,BBWAA,226,170,215,Y,Player,None
mathech01,1936,BBWAA,226,170,205,Y,Player,None
johnswa01,1936,BBWAA,226,170,189,Y,Player,None


In [25]:
%sql SELECT * FROM people LIMIT 5

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
5 rows affected.


playerid,birthyear,birthmonth,birthday,birthcountry,birthstate,birthcity,deathyear,deathmonth,deathday,deathcountry,deathstate,deathcity,namefirst,namelast,namegiven,weight,height,bats,throws,debut,finalgame,retroid,bbrefid
aardsda01,1981,12,27,USA,CO,Denver,None,None,None,None,None,None,David,Aardsma,David Allan,215,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01
aaronha01,1934,2,5,USA,AL,Mobile,None,None,None,None,None,None,Hank,Aaron,Henry Louis,180,72.0,R,R,1954-04-13,1976-10-03,aaroh101,aaronha01
aaronto01,1939,8,5,USA,AL,Mobile,1984,8,16,USA,GA,Atlanta,Tommie,Aaron,Tommie Lee,190,75.0,R,R,1962-04-10,1971-09-26,aarot101,aaronto01
aasedo01,1954,9,8,USA,CA,Orange,None,None,None,None,None,None,Don,Aase,Donald William,190,75.0,R,R,1977-07-26,1990-10-03,aased001,aasedo01
abadan01,1972,8,25,USA,FL,Palm Beach,None,None,None,None,None,None,Andy,Abad,Fausto Andres,184,73.0,L,L,2001-09-10,2006-04-13,abada001,abadan01


In [26]:
%sql SELECT * FROM schools LIMIT 5

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
5 rows affected.


schoolid,schoolname,schoolcity,schoolstate,schoolnick
abilchrist,Abilene Christian University,Abilene,TX,USA
adelphi,Adelphi University,Garden City,NY,USA
adrianmi,Adrian College,Adrian,MI,USA
akron,University of Akron,Akron,OH,USA
alabama,University of Alabama,Tuscaloosa,AL,USA


In [27]:
%sql SELECT * FROM collegeplaying LIMIT 5

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
5 rows affected.


playerid,schoolid,yearid
aardsda01,pennst,2001
aardsda01,rice,2002
aardsda01,rice,2003
abadan01,gamiddl,1992
abadan01,gamiddl,1993


In [28]:
%%sql result_1c <<
DROP VIEW IF EXISTS inducted_hof_ca;
CREATE VIEW inducted_hof_ca AS
SELECT people.namefirst as namefirst, people.namelast as namelast, collegeplaying.playerid, schools.schoolid, halloffame.yearid
FROM people, halloffame, schools, collegeplaying
WHERE people.playerid = halloffame.playerid and collegeplaying.playerid = people.playerid and collegeplaying.schoolid = schools.schoolid  and halloffame.inducted = 'Y' AND schools.schoolstate = 'CA';



SELECT * FROM inducted_hof_ca LIMIT 100;



 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Done.
23 rows affected.
Returning data to local variable result_1c


In [29]:
result_1c

namefirst,namelast,playerid,schoolid,yearid
Jackie,Robinson,robinja02,ucla,1962
Harry,Hooper,hoopeha01,stmarysca,1971
Joe,Morgan,morgajo02,camerri,1990
Tom,Seaver,seaveto01,cafrecc,1992
Tom,Seaver,seaveto01,usc,1992
Ozzie,Smith,smithoz01,calpoly,2002
Ozzie,Smith,smithoz01,calpoly,2002
Ozzie,Smith,smithoz01,calpoly,2002
Ozzie,Smith,smithoz01,calpoly,2002
Tony,Gwynn,gwynnto01,sandiegost,2007


In [30]:
# Do not delete/edit this cell
result_1c.DataFrame().sort_values(['playerid', 'schoolid', 'yearid']).to_csv('results/result_1c.csv', index=False)

In [31]:
grader.check("q1c")

q1c results: All test cases passed!

### Question 1d:
For this question, we want to compute the count of players who were inducted into the Hall of Fame and played baseball at a college in California for each `schoolid` and `yearid` combination ordered by ascending `yearid`.

#### Question 1di:
Write two queries that accomplish this task -- one query using the view you created in the `Question 1c` and one query that does not use the view, common table expressions (CTEs), or any sub-queries.

In [32]:
%sql select * from inducted_hof_ca limit 10

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
10 rows affected.


namefirst,namelast,playerid,schoolid,yearid
Jackie,Robinson,robinja02,ucla,1962
Harry,Hooper,hoopeha01,stmarysca,1971
Joe,Morgan,morgajo02,camerri,1990
Tom,Seaver,seaveto01,cafrecc,1992
Tom,Seaver,seaveto01,usc,1992
Ozzie,Smith,smithoz01,calpoly,2002
Ozzie,Smith,smithoz01,calpoly,2002
Ozzie,Smith,smithoz01,calpoly,2002
Ozzie,Smith,smithoz01,calpoly,2002
Tony,Gwynn,gwynnto01,sandiegost,2007


In [33]:
%%sql with_view <<
SELECT count(*), schoolid, yearid
FROM inducted_hof_ca
GROUP BY schoolid, yearid
ORDER BY yearid ASC;




 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
13 rows affected.
Returning data to local variable with_view


In [34]:
%%sql without_view <<
SELECT COUNT(*), schools.schoolid, halloffame.yearid
FROM people INNER JOIN halloffame ON people.playerid = halloffame.playerid 
INNER JOIN collegeplaying ON collegeplaying.playerid= people.playerid  INNER JOIN schools ON collegeplaying.schoolid = schools.schoolid 
WHERE halloffame.inducted = 'Y' AND schools.schoolstate = 'CA'
GROUP BY schools.schoolid, halloffame.yearid
ORDER BY yearid ASC;



 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
13 rows affected.
Returning data to local variable without_view


In [35]:
without_view

count,schoolid,yearid
1,ucla,1962
1,stmarysca,1971
1,camerri,1990
1,cafrecc,1992
1,usc,1992
4,calpoly,2002
3,sandiegost,2007
1,capasad,2008
2,sandiegost,2010
1,calavco,2011


In [36]:
# Do not delete/edit this cell
with_view.DataFrame().sort_values(['schoolid', 'yearid']).to_csv('results/result_1di_view.csv', index=False)
without_view.DataFrame().sort_values(['schoolid', 'yearid']).to_csv('results/result_1di_no_view.csv', index=False)

In [37]:
grader.check("q1di")

q1di results: All test cases passed!

#### Question 1dii:
Fill in your queries from `Question 1di` and inspect the query plans for the two queries. Record the execution time and cost for each query.

In [38]:
with_view_query_str = "SELECT count(*), schoolid, yearid FROM inducted_hof_ca GROUP BY schoolid, yearid ORDER BY yearid ASC"
!psql -h localhost -d baseball -c "explain analyze $with_view_query_str"

                                                                             QUERY PLAN                                                                              
---------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=325.71..325.73 rows=1 width=60) (actual time=4.984..4.994 rows=13 loops=1)
   Group Key: halloffame.yearid, schools.schoolid
   ->  Sort  (cost=325.71..325.71 rows=1 width=52) (actual time=4.975..4.978 rows=23 loops=1)
         Sort Key: halloffame.yearid, schools.schoolid
         Sort Method: quicksort  Memory: 26kB
         ->  Nested Loop  (cost=0.85..325.70 rows=1 width=52) (actual time=1.893..4.915 rows=23 loops=1)
               ->  Nested Loop  (cost=0.57..319.95 rows=19 width=52) (actual time=0.233..4.458 rows=137 loops=1)
                     ->  Nested Loop  (cost=0.29..262.79 rows=21 width=80) (actual time=0.112..2.779 rows=3

In [39]:
without_view_query_str = "SELECT COUNT(*), schools.schoolid, halloffame.yearid FROM people INNER JOIN halloffame ON people.playerid = halloffame.playerid INNER JOIN collegeplaying ON collegeplaying.playerid= people.playerid  INNER JOIN schools ON collegeplaying.schoolid = schools.schoolid WHERE halloffame.inducted = 'Y' AND schools.schoolstate = 'CA' GROUP BY schools.schoolid, halloffame.yearid ORDER BY yearid ASC"
!psql -h localhost -d baseball -c"explain analyze $without_view_query_str"

                                                                             QUERY PLAN                                                                              
---------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=325.71..325.73 rows=1 width=60) (actual time=4.747..4.757 rows=13 loops=1)
   Group Key: halloffame.yearid, schools.schoolid
   ->  Sort  (cost=325.71..325.71 rows=1 width=52) (actual time=4.738..4.740 rows=23 loops=1)
         Sort Key: halloffame.yearid, schools.schoolid
         Sort Method: quicksort  Memory: 26kB
         ->  Nested Loop  (cost=0.85..325.70 rows=1 width=52) (actual time=1.852..4.692 rows=23 loops=1)
               ->  Nested Loop  (cost=0.57..319.95 rows=19 width=52) (actual time=0.177..4.237 rows=137 loops=1)
                     ->  Nested Loop  (cost=0.29..262.79 rows=21 width=80) (actual time=0.065..2.653 rows=3

In [40]:
with_view_cost = 518.87
with_view_timing = 6.236
without_view_cost = 518.87
without_view_timing = 6.152

In [41]:
grader.check("q1dii")

q1dii results: All test cases passed!

### Question 1e:
#### Question 1ei:
Now, let's try creating a materialized view named `inducted_hof_ca_mat` instead of the regular view from `Question 1c`.

In [42]:
%%sql inducted_hof_ca_materialized <<
DROP MATERIALIZED VIEW IF EXISTS inducted_hof_ca_mat;
CREATE MATERIALIZED VIEW inducted_hof_ca_mat AS
SELECT people.namefirst as namefirst, people.namelast as namelast, collegeplaying.playerid, schools.schoolid, halloffame.yearid
FROM people, halloffame, schools, collegeplaying
WHERE people.playerid = halloffame.playerid and collegeplaying.playerid = people.playerid and collegeplaying.schoolid = schools.schoolid  and halloffame.inducted = 'Y' AND schools.schoolstate = 'CA';
SELECT * FROM inducted_hof_ca_mat;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
23 rows affected.
23 rows affected.
Returning data to local variable inducted_hof_ca_materialized


In [43]:
# Do not delete/edit this cell
inducted_hof_ca_materialized.DataFrame().sort_values(['playerid', 'schoolid', 'yearid']).to_csv('results/result_1ei.csv', index=False)

In [44]:
grader.check("q1ei")

q1ei results: All test cases passed!

#### Question 1eii:

Now, rewrite the query from `Question 1d` to use the materialized view `inducted_hof_ca_mat`.

In [45]:
%%sql with_materialized_view <<
SELECT count(*), schoolid, yearid
FROM inducted_hof_ca_mat
GROUP BY schoolid, yearid
ORDER BY yearid ASC

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
13 rows affected.
Returning data to local variable with_materialized_view


In [46]:
# Do not delete/edit this cell
with_materialized_view.DataFrame().sort_values(['schoolid', 'yearid']).to_csv('results/result_1eii.csv', index=False)

In [47]:
grader.check("q1eii")

q1eii results: All test cases passed!

#### Question 1eiii:
Inspect the query plan and record the execution time and cost of the query that uses the materialized view.

In [48]:
with_materialized_view_query_str = "SELECT count(*), schoolid, yearid FROM inducted_hof_ca_mat GROUP BY schoolid, yearid ORDER BY yearid ASC"
!psql -h localhost -d baseball -c "explain analyze $with_materialized_view_query_str"

                                                          QUERY PLAN                                                          
------------------------------------------------------------------------------------------------------------------------------
 Sort  (cost=23.67..24.17 rows=200 width=60) (actual time=0.065..0.066 rows=13 loops=1)
   Sort Key: yearid
   Sort Method: quicksort  Memory: 25kB
   ->  HashAggregate  (cost=14.03..16.02 rows=200 width=60) (actual time=0.022..0.025 rows=13 loops=1)
         Group Key: yearid, schoolid
         Batches: 1  Memory Usage: 40kB
         ->  Seq Scan on inducted_hof_ca_mat  (cost=0.00..12.30 rows=230 width=52) (actual time=0.005..0.006 rows=23 loops=1)
 Planning Time: 0.344 ms
 Execution Time: 0.201 ms
(9 rows)



In [49]:
with_materialized_view_cost = 24.17
with_materialized_view_timing = 0.195

In [50]:
grader.check("q1eiii")

q1eiii results: All test cases passed!

#### Question 1eiv:
Given your findings from inspecting the query plans, as well as your understanding of views and materialized views from lectures, answer the following question. Assign the variable `q1e_part4` to a list of all statements which are true.

1. Consider the following statements:
    1. Views will reduce the execution time and the cost of a query.
    1. Views will reduce the execution time of a query, but not the cost.
    1. Views will reduce the cost of a query, but not the execution time.
    1. Materialized views reduce the execution time and the cost of a query.
    1. Materialized views reduce the execution time, but not cost of a query.
    1. Materialized views reduce the cost of a query, but not the execution time.
    1. Materialized views will result in the same query plan as a query using views.
    1. Materialized views and views take the same time to create.
    1. Materialized views take less time to create than a view.
    1. Materialized views take more time to create than a view.
    
*Note:* Your answer should look like `q1e_part4 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q1eiv
points: 2
-->

In [51]:
q1e_part4 = ['B', 'D', 'J']

In [52]:
grader.check("q1eiv")

q1eiv results: All test cases passed!

<!-- BEGIN QUESTION -->

#### Question 1ev:

Explain your answer to the previous part based on your knowledge from lectures, and details from the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q1ev
manual: true
points: 2
-->

Views will reduce the execution time of a query because they'll store temporary results that can speed up resulting queries. However, since the views store results, they are still increasing the cost. Materialized views reduce execution time and the cost of a query because every time the database runs its query, it will only run it once and store the results in a newly created database. In a materialized view, it does not recalculate the results every time the query is run because the values are stored, which makes the cost less and the runtime faster. Since storing values, a materialized view takes longer to create than a standard view. 


<!-- END QUESTION -->



## Question 2: Predicate Pushdown
In this question, we will explore the impact of predicates on a query's execution, particularly inspecting when the optimizer applies predicates.

### Question 2a:
Recall the `inducted_hof_ca` created in `Question 1c`. Inspect the query plan for a query that that gets all rows from the view, and record the execution time and cost.

In [53]:
query_view_str = "SELECT people.namefirst as namefirst, people.namelast as namelast, collegeplaying.playerid, schools.schoolid, halloffame.yearid FROM people, halloffame, schools, collegeplaying WHERE people.playerid = halloffame.playerid and collegeplaying.playerid = people.playerid and collegeplaying.schoolid = schools.schoolid  and halloffame.inducted = 'Y' AND schools.schoolstate = 'CA';"
!psql -h localhost -d baseball -c "explain analyze $query_view_str"

                                                                       QUERY PLAN                                                                        
---------------------------------------------------------------------------------------------------------------------------------------------------------
 Nested Loop  (cost=0.85..325.70 rows=1 width=326) (actual time=1.896..4.709 rows=23 loops=1)
   ->  Nested Loop  (cost=0.57..319.95 rows=19 width=326) (actual time=0.179..4.252 rows=137 loops=1)
         ->  Nested Loop  (cost=0.29..262.79 rows=21 width=316) (actual time=0.062..2.687 rows=323 loops=1)
               ->  Seq Scan on halloffame  (cost=0.00..96.39 rows=21 width=42) (actual time=0.029..0.676 rows=323 loops=1)
                     Filter: ((inducted)::text = 'Y'::text)
                     Rows Removed by Filter: 3868
               ->  Index Scan using master_pkey on people  (cost=0.29..7.92 rows=1 width=274) (actual time=0.006..0.006 rows=1 loops=323)
                 

In [54]:
query_view_cost = 527.38
query_view_timing = 15.935

In [55]:
grader.check("q2a")

q2a results: All test cases passed!

### Question 2b:
Now, add a filter to only return rows from `inducted_hof_ca` where the year is later than 2010. Inspect the query plan and record the execution time and cost.

In [56]:
%sql SELECT * from inducted_hof_ca limit 10

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
10 rows affected.


namefirst,namelast,playerid,schoolid,yearid
Jackie,Robinson,robinja02,ucla,1962
Harry,Hooper,hoopeha01,stmarysca,1971
Joe,Morgan,morgajo02,camerri,1990
Tom,Seaver,seaveto01,cafrecc,1992
Tom,Seaver,seaveto01,usc,1992
Ozzie,Smith,smithoz01,calpoly,2002
Ozzie,Smith,smithoz01,calpoly,2002
Ozzie,Smith,smithoz01,calpoly,2002
Ozzie,Smith,smithoz01,calpoly,2002
Tony,Gwynn,gwynnto01,sandiegost,2007


In [57]:
%%sql result_2b <<
select * from inducted_hof_ca 
where yearid > 2010;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
8 rows affected.
Returning data to local variable result_2b


In [58]:
result_2b

namefirst,namelast,playerid,schoolid,yearid
Pat,Gillick,gillipa99,calavco,2011
Pat,Gillick,gillipa99,usc,2011
Pat,Gillick,gillipa99,usc,2011
Randy,Johnson,johnsra05,usc,2015
Randy,Johnson,johnsra05,usc,2015
Randy,Johnson,johnsra05,usc,2015
Trevor,Hoffman,hoffmtr01,cacypre,2018
Trevor,Hoffman,hoffmtr01,cacypre,2018


In [59]:
query_view_with_filter_str = "select * from inducted_hof_ca where yearid > 2010"
!psql -h localhost -d baseball -c "explain analyze $query_view_with_filter_str"

                                                                       QUERY PLAN                                                                       
--------------------------------------------------------------------------------------------------------------------------------------------------------
 Nested Loop  (cost=0.85..185.87 rows=1 width=326) (actual time=0.709..1.401 rows=8 loops=1)
   ->  Nested Loop  (cost=0.57..184.05 rows=6 width=326) (actual time=0.679..1.249 rows=27 loops=1)
         ->  Nested Loop  (cost=0.29..165.00 rows=7 width=316) (actual time=0.561..0.941 rows=31 loops=1)
               ->  Seq Scan on halloffame  (cost=0.00..106.86 rows=7 width=42) (actual time=0.519..0.563 rows=31 loops=1)
                     Filter: ((yearid > 2010) AND ((inducted)::text = 'Y'::text))
                     Rows Removed by Filter: 4160
               ->  Index Scan using master_pkey on people  (cost=0.29..8.30 rows=1 width=274) (actual time=0.011..0.011 rows=1 loops=31)
    

In [60]:
query_view_with_filter_cost = 218.51
query_view_with_filter_timing = 1.552

In [61]:
grader.check("q2b")

q2b results: All test cases passed!

### Question 2c:
Given your findings from inspecting the query plans of queries from `Question 2a-b`, answer the following question. Assign the variable `q2c` to a list of all statements which are true.

1. Consider the following statements:
    1. Adding a filter lowered the cost.
    1. Adding a filter increased the cost.
    1. Adding a filter did not change the cost.
    1. Adding a filter increased the execution time.
    1. Adding a filter decreased the execution time.
    1. Adding a filter did not change the execution time.
    1. No statement is true.
    
    
**Note:** Your answer should look like `q2c = ['A', 'B']`

<!--
BEGIN QUESTION
name: q2c
points: 2
-->

In [62]:
q2c = ['A', 'E']

In [63]:
grader.check("q2c")

q2c results: All test cases passed!

<!-- BEGIN QUESTION -->

#### Question 2d:

Explain your answer based on your knowledge from lectures, and details from the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q2d
manual: true
points: 2
-->

Adding a filter lowered the cost because it only selected a small amount of data that it had to go through which made the query run faster. In question 2, when we wrote a query to get the all the rows with no filter, the cost was over 500 and the runtime was 15. When we added the view, it had a smaller amount of data to run through which means that the cost was lower and the runtime was much lower.

<!-- END QUESTION -->



## Question 3: Join Approaches

In this question, we'll explore different join approaches (Nested Loop Join, Merge Join, Hash Join) and discuss how the query optimizer picks the best approach.

### Question 3a:
Perform an inner join on the `people` and `collegeplaying` tables on the `playerid` column. Project all columns.

In [64]:
%%sql q3a_join <<
select * from people inner join collegeplaying on people.playerid = collegeplaying.playerid



 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
17350 rows affected.
Returning data to local variable q3a_join


Inspect the query plan for the query you wrote above.

In [65]:
q3a_query_str = "select * from people inner join collegeplaying on people.playerid = collegeplaying.playerid"
!psql -h localhost -d baseball -c "explain analyze $q3a_query_str"

                                                          QUERY PLAN                                                           
-------------------------------------------------------------------------------------------------------------------------------
 Hash Join  (cost=503.38..1562.91 rows=17350 width=1719) (actual time=5.011..16.392 rows=17350 loops=1)
   Hash Cond: ((people.playerid)::text = (collegeplaying.playerid)::text)
   ->  Seq Scan on people  (cost=0.00..619.70 rows=19370 width=1698) (actual time=0.022..2.761 rows=19370 loops=1)
   ->  Hash  (cost=286.50..286.50 rows=17350 width=21) (actual time=4.762..4.765 rows=17350 loops=1)
         Buckets: 32768  Batches: 1  Memory Usage: 1203kB
         ->  Seq Scan on collegeplaying  (cost=0.00..286.50 rows=17350 width=21) (actual time=0.023..1.446 rows=17350 loops=1)
 Planning Time: 0.929 ms
 Execution Time: 17.184 ms
(8 rows)



Given your findings from inspecting the query plan, answer the following question. Assign the variable `q3a` to a list of all the options which are true for the proposed statement.

1. Which join approach did the query optimizer choose?
    1. Nested Loop Join
    1. Merge Join
    1. Hash Join
    1. None of the Above

**Note:** Your answer should be formatted as follows: `q3a = ['A', 'B']`

<!--
BEGIN QUESTION
name: q3a
points: 1
-->

In [66]:
q3a = ['B']

In [67]:
grader.check("q3a")

q3a results: All test cases passed!

### Question 3b

Similar to Question 3a, perform an inner join on the `people` and `collegeplaying` tables on the `playerid` column. Project all columns.

In addition, **sort your output by `playerid`.**

In [68]:
%%sql q3b_join <<
select * from people inner join collegeplaying on people.playerid = collegeplaying.playerid
ORDER BY collegeplaying.playerid


 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
17350 rows affected.
Returning data to local variable q3b_join


Inspect the query plan for the query you wrote above.

In [69]:
q3b_query_str = "select * from people inner join collegeplaying on people.playerid = collegeplaying.playerid ORDER BY collegeplaying.playerid"
!psql -h localhost -d baseball -c "explain analyze $q3b_query_str"

                                                                        QUERY PLAN                                                                        
----------------------------------------------------------------------------------------------------------------------------------------------------------
 Merge Join  (cost=0.57..3184.67 rows=17350 width=1719) (actual time=0.083..22.710 rows=17350 loops=1)
   Merge Cond: ((people.playerid)::text = (collegeplaying.playerid)::text)
   ->  Index Scan using master_pkey on people  (cost=0.29..2298.84 rows=19370 width=1698) (actual time=0.027..5.513 rows=19368 loops=1)
   ->  Index Only Scan using collegeplaying_pkey on collegeplaying  (cost=0.29..620.54 rows=17350 width=21) (actual time=0.052..2.964 rows=17350 loops=1)
         Heap Fetches: 0
 Planning Time: 0.893 ms
 Execution Time: 23.587 ms
(7 rows)



Given your findings from inspecting the query plan, answer the following question. Assign the variable `q3b` to a list of all the options which are true for the proposed statement.

1. Which join approach did the query optimizer choose?
    1. Nested Loop Join
    1. Merge Join
    1. Hash Join
    1. None of the Above

**Note:** Your answer should be formatted as follows: `q3b = ['A', 'B']`

<!--
BEGIN QUESTION
name: q3b
points: 1
-->

In [70]:
q3b = ['B']

In [71]:
grader.check("q3b")

q3b results: All test cases passed!

### Question 3c
Write a query to retrieve all possible player pair combinations. Project all columns, but **limit to 1000 rows** to ensure your query doesn't take an exorbitant amount of time to run.

**Hint:** You can do this by performing an inner join of the `people` table on itself with an inequality condition.

In [72]:
%%sql q3c_join <<
select * from people as p1 inner join people p2 on p1.playerid != p2.playerid LIMIT 1000

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
1000 rows affected.
Returning data to local variable q3c_join


Inspect the query plan for the query you wrote above.

In [73]:
q3c_query_str = "select * from people as p1 inner join people p2 on p1.playerid != p2.playerid LIMIT 1000"
!psql -h localhost -d baseball -c "explain analyze $q3c_query_str"

                                                           QUERY PLAN                                                           
--------------------------------------------------------------------------------------------------------------------------------
 Limit  (cost=0.00..15.00 rows=1000 width=292) (actual time=0.038..1.062 rows=1000 loops=1)
   ->  Nested Loop  (cost=0.00..5629241.33 rows=375177530 width=292) (actual time=0.036..0.991 rows=1000 loops=1)
         Join Filter: ((p1.playerid)::text <> (p2.playerid)::text)
         Rows Removed by Join Filter: 1
         ->  Seq Scan on people p1  (cost=0.00..619.70 rows=19370 width=146) (actual time=0.020..0.021 rows=1 loops=1)
         ->  Materialize  (cost=0.00..716.55 rows=19370 width=146) (actual time=0.009..0.527 rows=1001 loops=1)
               ->  Seq Scan on people p2  (cost=0.00..619.70 rows=19370 width=146) (actual time=0.001..0.109 rows=1001 loops=1)
 Planning Time: 0.871 ms
 Execution Time: 1.236 ms
(9 rows)



Given your findings from inspecting the query plan, answer the following question. Assign the variable `q3c` to a list of all the options which are true for the proposed statement.

1. Which join approach did the query optimizer choose?
    1. Nested Loop Join
    1. Merge Join
    1. Hash Join
    1. None of the Above

**Note:** Your answer should be formatted as follows: `q3c = ['A', 'B']`

<!--
BEGIN QUESTION
name: q3c
points: 1
-->

In [74]:
q3c = ['A']

In [75]:
grader.check("q3c")

q3c results: All test cases passed!

<!-- BEGIN QUESTION -->

### Question 3d
Please discuss your findings in the previous parts. In particular, we are interested in hearing why you think the query optimizer chooses the ultimate join approach in each of the above three scenarios. Feel free to discuss the pros and cons of each join approach as well.

If you feel stuck, here are some things to consider: Does a non-equijoin constrain us to certain join approaches? What's an added benefit in regards to the output of merge join?

<!--
BEGIN QUESTION
name: q3d
manual: true
points: 3
-->

In question 3a, the query optimizer used a hash join because a hash joined is used on tables that are already not sorted. In this question, we only selected columns to do an inner join but neither table had sorted columns. The hash join was used to sort the data from the two tables and then joins the two tables. In question 3b, a merge join was used because the two tables are already being sorted by the "order by" method. The merge join is better than the hash join in this sceanario because the two tables are already being sorted and if the hash method was used then the excecution time and the cost would be much higher. The merge method was the fastest method to use in this case since the two tables are sorted. In question 3c, the nested loop join was used because the table is smaller and you are cross joining in the same table. The nested loop join is the fastest join and least costly in this situation because there is an inner join in the same table. The hash join and the merge join is unnesscary in this case because there is no need to sort values or merge two large quantity of tables. Since the data in one table is small and there is a cross join the nested for loop is the best one to use.

<!-- END QUESTION -->



## Question 4: Indexes

### Question 4a:
Write a query that outputs the `playerid` and average `salary` for each player that only batted in 10 games (the number of games in which a player batted can be found in the `g_batting` column of the `appearances` table). Your query should join the `salaries` and `appearances` table on all the common columns `yearid`, `teamid`, and `playerid`, so feel free to use a natural join.

In [76]:
%sql select * from salaries LIMIT 5

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
5 rows affected.


yearid,teamid,lgid,playerid,salary
1985,ATL,NL,barkele01,870000.0
1985,ATL,NL,bedrost01,550000.0
1985,ATL,NL,benedbr01,545000.0
1985,ATL,NL,campri01,633333.0
1985,ATL,NL,ceronri01,625000.0


In [77]:
%sql select * from appearances LIMIT 5

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
5 rows affected.


yearid,teamid,lgid,playerid,g_all,gs,g_batting,g_defense,g_p,g_c,g_1b,g_2b,g_3b,g_ss,g_lf,g_cf,g_rf,g_of,g_dh,g_ph,g_pr
1871,TRO,NA,abercda01,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0
1871,RC1,NA,addybo01,25,25,25,25,0,0,0,22,0,3,0,0,0,0,0,0,0
1871,CL1,NA,allisar01,29,29,29,29,0,0,0,2,0,0,0,29,0,29,0,0,0
1871,WS3,NA,allisdo01,27,27,27,27,0,27,0,0,0,0,0,0,0,0,0,0,0
1871,RC1,NA,ansonca01,25,25,25,25,0,5,1,2,20,0,1,0,0,1,0,0,0


In [78]:
%%sql result_4a <<
SELECT salaries.playerid, avg(salaries.salary)
FROM salaries NATURAL JOIN appearances 
WHERE g_batting = 10
GROUP BY salaries.playerid;


 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
134 rows affected.
Returning data to local variable result_4a


In [79]:
# Do not delete/edit this cell
result_4a.DataFrame().sort_values('playerid').to_csv('results/result_4a.csv', index=False)

In [80]:
grader.check("q4ai")

q4ai results: All test cases passed!

Inspect the query plan and record the execution time and cost.

In [81]:
result_4a_query_str = "SELECT salaries.playerid, avg(salaries.salary) FROM salaries NATURAL JOIN appearances WHERE g_batting = 10 GROUP BY salaries.playerid;"
!psql -h localhost -d baseball -c "explain analyze $result_4a_query_str"

                                                                                                                     QUERY PLAN                                                                                                                     
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=3637.58..3637.61 rows=1 width=17) (actual time=23.086..23.154 rows=134 loops=1)
   Group Key: salaries.playerid
   ->  Sort  (cost=3637.58..3637.59 rows=1 width=17) (actual time=23.074..23.084 rows=138 loops=1)
         Sort Key: salaries.playerid
         Sort Method: quicksort  Memory: 35kB
         ->  Hash Join  (cost=2900.80..3637.57 rows=1 width=17) (actual time=14.489..22.877 rows=138 loops=1)
               Hash Cond: ((salaries.yearid = appearances.yearid) AND ((salaries.te

In [82]:
result_4a_cost = 3635.73
result_4a_timing = 26.048 

In [83]:
grader.check("q4aii")

q4aii results: All test cases passed!

### Question 4b:
Add an index with name `g_batting_idx` on the `g_batting` column of the `appearances` table.

In [84]:
%%sql result_4b <<
create index g_batting_idx on appearances(g_batting)

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4b


Now, re-inspect the query plan of the query from `Question 4a` and record its execution time and cost.

In [85]:
!psql -h localhost -d baseball -c "explain analyze $result_4a_query_str"

                                                                                                                     QUERY PLAN                                                                                                                     
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=2388.09..2388.11 rows=1 width=17) (actual time=11.831..11.904 rows=134 loops=1)
   Group Key: salaries.playerid
   ->  Sort  (cost=2388.09..2388.09 rows=1 width=17) (actual time=11.820..11.830 rows=138 loops=1)
         Sort Key: salaries.playerid
         Sort Method: quicksort  Memory: 35kB
         ->  Hash Join  (cost=1651.30..2388.08 rows=1 width=17) (actual time=3.198..11.632 rows=138 loops=1)
               Hash Cond: ((salaries.yearid = appearances.yearid) AND ((salaries.tea

In [86]:
result_4b_cost = 2360.15
result_4b_timing = 14.405

In [87]:
grader.check("q4b")

q4b results: All test cases passed!

In the following question, we will explore adding a different index and evaluating the query from `Question 4a`. To avoid any interference by the `g_batting_idx` index, **drop the index before moving onto the next question.**

In [88]:
%sql drop index g_batting_idx 

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

### Question 4c:
Write a query to add an index with name `salary_idx` on the `salary` column of the `salaries` table.

In [89]:
%%sql result_4c <<
create index salary_idx on salaries(salary)

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4c


Re-inspect the query plan of the query from the `Question 4a` and record its execution time and cost.

In [90]:
!psql -h localhost -d baseball -c "explain analyze $result_4a_query_str"

                                                                                                                     QUERY PLAN                                                                                                                     
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=3637.58..3637.61 rows=1 width=17) (actual time=21.713..21.780 rows=134 loops=1)
   Group Key: salaries.playerid
   ->  Sort  (cost=3637.58..3637.59 rows=1 width=17) (actual time=21.700..21.710 rows=138 loops=1)
         Sort Key: salaries.playerid
         Sort Method: quicksort  Memory: 35kB
         ->  Hash Join  (cost=2900.80..3637.57 rows=1 width=17) (actual time=13.545..21.520 rows=138 loops=1)
               Hash Cond: ((salaries.yearid = appearances.yearid) AND ((salaries.te

In [91]:
result_4c_cost = 3635.79
result_4c_timing = 26.681

In [92]:
grader.check("q4c")

q4c results: All test cases passed!

### Question 4d:
#### Question 4di:
Given your findings from inspecting the query plans with no indexes, an index on `g_batting`, and an index on `salary`, answer the following question. Assign the variable `q4d_part1` to a list of all options which are true for the proposed statement.

1. Consider the following statements:
    1. Adding the `g_batting` index did not have a significant impact on the query execution time and cost.
    1. Adding the `g_batting` index did have a significant impact on the query execution time, but not the cost.
    1. Adding the `g_batting` index did have a significant impact on the query cost, but not the execution time.
    1. Adding the `g_batting` index did have a significant impact on the query cost and execution time.
    1. Adding the `salary` index did not have a significant impact on the query execution time and cost.
    1. Adding the `salary` index did have a significant impact on the query execution time, but not the cost.
    1. Adding the `salary` index did have a significant impact on the query cost, but not the execution time.
    1. Adding the `salary` index did have a significant impact on the query cost and execution time.

**Note:** Your answer should be formatted as follows: `q4d_part1 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q4di
points: 2
-->

In [93]:
q4d_part1 = ['D', 'H' ]

In [94]:
grader.check("q4di")

q4di results: All test cases passed!

<!-- BEGIN QUESTION -->

#### Question 4dii:

Explain your answer based on your knowledge from lectures, and details from inspecting the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q4dii
manual: true
points: 2
-->

Adding an index can make the execution time either become really fast or slow. The index can make it become a pointers of where the data is stored in the tables which makes it faster to join. Indexes can also make the execution time slower since it has more indexes, the query would have to do more processing that would increase the cost. Adding indexes can make the costs go higher since it is creating more data and it makes the query go longer

<!-- END QUESTION -->



In the following question, we will further explore the impact of indexes on query performance. To avoid any interference by the `salary_idx` index, please drop the index before moving onto the next question.

In [95]:
%%sql 
drop index salary_idx

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

### Question 4e:
#### Question 4ei:
Write a query that finds the `playerid`, `yearid`, and `salary` for each player that had played 10 games and batted in 10 games (the number of games in which a player played can be found in the `g_all` column of the `appearances` table). Your query should join the `salaries` and `appearances` table on all the common columns `yearid`, `teamid`, and `playerid`, so feel free to use a natural join.

In [96]:
%sql select * from appearances limit 10

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
10 rows affected.


yearid,teamid,lgid,playerid,g_all,gs,g_batting,g_defense,g_p,g_c,g_1b,g_2b,g_3b,g_ss,g_lf,g_cf,g_rf,g_of,g_dh,g_ph,g_pr
1871,TRO,NA,abercda01,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0
1871,RC1,NA,addybo01,25,25,25,25,0,0,0,22,0,3,0,0,0,0,0,0,0
1871,CL1,NA,allisar01,29,29,29,29,0,0,0,2,0,0,0,29,0,29,0,0,0
1871,WS3,NA,allisdo01,27,27,27,27,0,27,0,0,0,0,0,0,0,0,0,0,0
1871,RC1,NA,ansonca01,25,25,25,25,0,5,1,2,20,0,1,0,0,1,0,0,0
1871,FW1,NA,armstbo01,12,12,12,12,0,0,0,0,0,0,0,11,1,12,0,0,0
1871,RC1,NA,barkeal01,1,1,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0
1871,BS1,NA,barnero01,31,31,31,31,0,0,0,16,0,15,0,0,0,0,0,0,0
1871,FW1,NA,barrebi01,1,1,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0
1871,BS1,NA,barrofr01,18,17,18,18,0,0,0,1,0,0,13,0,4,17,0,0,0


In [97]:
%%sql result_4ei <<
SELECT salaries.playerid, salaries.yearid, avg(salaries.salary)
FROM salaries NATURAL JOIN appearances 
WHERE g_all = 10 and g_batting = 10
GROUP BY salaries.playerid,salaries.yearid;


 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
120 rows affected.
Returning data to local variable result_4ei


In [98]:
# Do not delete/edit this cell
result_4ei.DataFrame().sort_values(['playerid', 'yearid']).to_csv('results/result_4ei.csv', index=False)

In [99]:
grader.check("q4ei_part1")

q4ei_part1 results: All test cases passed!

Inspect the query plan and record the execution time and cost.

In [100]:
result_4ei_query_str = "SELECT salaries.playerid, salaries.yearid, avg(salaries.salary) FROM salaries NATURAL JOIN appearances  WHERE g_all = 10 and g_batting = 10 GROUP BY salaries.playerid,salaries.yearid"
!psql -h localhost -d baseball -c "explain analyze $result_4ei_query_str"

                                                                                                      QUERY PLAN                                                                                                       
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=3308.73..3308.75 rows=1 width=21) (actual time=17.174..17.233 rows=120 loops=1)
   Group Key: salaries.playerid, salaries.yearid
   ->  Sort  (cost=3308.73..3308.73 rows=1 width=21) (actual time=17.163..17.170 rows=120 loops=1)
         Sort Key: salaries.playerid, salaries.yearid
         Sort Method: quicksort  Memory: 34kB
         ->  Nested Loop  (cost=0.29..3308.72 rows=1 width=21) (actual time=9.655..16.987 rows=120 loops=1)
               ->  Seq Scan on appearances  (cost=0.00..3133.84 rows=22 width=20) (actual time=0.018..13.628 

In [101]:
result_4ei_cost = 3300.44
result_4ei_timing = 22.220

In [102]:
grader.check("q4ei_part2")

q4ei_part2 results: All test cases passed!

#### Question 4eii:
Now, let's see the impact of adding an index on the `g_batting` column. Create an index on the `g_batting` column. Re-inspect the query from `Question 4ei` and record the execution time and cost.

In [103]:
%%sql result_4eii << 
create index g_batting_idx on appearances(g_batting)

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4eii


In [104]:
!psql -h localhost -d baseball -c "explain analyze $result_4ei_query_str"

                                                                                                      QUERY PLAN                                                                                                       
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=1801.70..1801.73 rows=1 width=21) (actual time=9.399..9.516 rows=120 loops=1)
   Group Key: salaries.playerid, salaries.yearid
   ->  Sort  (cost=1801.70..1801.71 rows=1 width=21) (actual time=9.388..9.401 rows=120 loops=1)
         Sort Key: salaries.playerid, salaries.yearid
         Sort Method: quicksort  Memory: 34kB
         ->  Nested Loop  (cost=18.94..1801.69 rows=1 width=21) (actual time=4.639..9.126 rows=120 loops=1)
               ->  Bitmap Heap Scan on appearances  (cost=18.65..1626.81 rows=22 width=20) (actual time=0.435..3.

In [105]:
result_4eii_with_index_cost = 1782.44
result_4eii_with_index_timing = 6.349

In [106]:
grader.check("q4eii")

q4eii results: All test cases passed!

In the following question, we will further explore the impact of indexes on query performance. To avoid any interference from the index on the `g_batting` column, drop the index before moving onto the next question.

In [107]:
%%sql 
drop index  g_batting_idx

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

#### Question 4eiii:
Write a query that finds the `playerid`, `yearid`, and `salary` for each player that had played 10 games __or__ batted in 10 games. Your query should join the `salaries` and `appearances` table on all the common columns `yearid`, `teamid`, and `playerid`, so feel free to use a natural join.

In [108]:
%%sql result_4eiii <<
SELECT salaries.playerid, salaries.yearid, avg(salaries.salary)
FROM salaries NATURAL JOIN appearances 
WHERE g_all = 10 or g_batting = 10
GROUP BY salaries.playerid,salaries.yearid;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
216 rows affected.
Returning data to local variable result_4eiii


In [109]:
result_4eiii

playerid,yearid,avg
abbotpa01,2004,600000.0
aceveal01,2010,435650.0
almanca01,2001,270000.0
alvarwi01,1991,100000.0
anderla02,1986,240000.0
ashbyan01,1992,109000.0
austije01,2002,205000.0
ayraubo01,1993,125000.0
baekch01,2008,392500.0
bankswi01,1995,240000.0


In [110]:
# Do not delete/edit this cell
result_4eiii.DataFrame().sort_values(['playerid', 'yearid']).to_csv('results/result_4eiii.csv', index=False)

In [111]:
grader.check("q4eiii_part1")

q4eiii_part1 results: All test cases passed!

In [112]:
result_4eiii_query_str = "SELECT salaries.playerid, salaries.yearid, avg(salaries.salary) FROM salaries NATURAL JOIN appearances  WHERE g_all = 10 or g_batting = 10 GROUP BY salaries.playerid,salaries.yearid"
!psql -h localhost -d baseball -c "explain analyze $result_4eiii_query_str"

                                                                                                                     QUERY PLAN                                                                                                                     
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=3930.72..3930.75 rows=1 width=21) (actual time=23.996..24.102 rows=216 loops=1)
   Group Key: salaries.playerid, salaries.yearid
   ->  Sort  (cost=3930.72..3930.73 rows=1 width=21) (actual time=23.983..23.996 rows=216 loops=1)
         Sort Key: salaries.playerid, salaries.yearid
         Sort Method: quicksort  Memory: 41kB
         ->  Hash Join  (cost=3193.94..3930.71 rows=1 width=21) (actual time=15.021..23.676 rows=216 loops=1)
               Hash Cond: ((salaries.yearid = app

In [113]:
result_4eiii_cost = 3929.39
result_4eiii_timing = 24.222

In [114]:
grader.check("q4eiii_part2")

q4eiii_part2 results: All test cases passed!

#### Question 4eiv
Now, let's see the impact of adding an index on `g_batting` column will have on the query. Re-create the index and re-inspect the query from `Question 4eiii` and record the execution time and cost.

In [115]:
%%sql result_4eiv << 
SELECT salaries.playerid, salaries.yearid, avg(salaries.salary)
FROM salaries NATURAL JOIN appearances 
WHERE g_all = 10 or g_batting = 10
GROUP BY salaries.playerid,salaries.yearid;
create index g_batting_idx on appearances(g_batting);


 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
216 rows affected.
Done.
Returning data to local variable result_4eiv


In [116]:
result_4eiii

playerid,yearid,avg
abbotpa01,2004,600000.0
aceveal01,2010,435650.0
almanca01,2001,270000.0
alvarwi01,1991,100000.0
anderla02,1986,240000.0
ashbyan01,1992,109000.0
austije01,2002,205000.0
ayraubo01,1993,125000.0
baekch01,2008,392500.0
bankswi01,1995,240000.0


In [117]:
!psql -h localhost -d baseball -c "explain analyze $result_4eiii_query_str"

                                                                                                                     QUERY PLAN                                                                                                                     
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=3930.72..3930.75 rows=1 width=21) (actual time=25.141..25.250 rows=216 loops=1)
   Group Key: salaries.playerid, salaries.yearid
   ->  Sort  (cost=3930.72..3930.73 rows=1 width=21) (actual time=25.127..25.142 rows=216 loops=1)
         Sort Key: salaries.playerid, salaries.yearid
         Sort Method: quicksort  Memory: 41kB
         ->  Hash Join  (cost=3193.94..3930.71 rows=1 width=21) (actual time=15.286..24.697 rows=216 loops=1)
               Hash Cond: ((salaries.yearid = app

In [118]:
result_4eiv_with_index_cost = 3929.39
result_4eiv_with_index_timing = 25.177

In [119]:
grader.check("q4eiv")

q4eiv results: All test cases passed!

In the following question, we will further explore the impact of indexes on query performance. To avoid any interference from the index on the `g_batting` column, drop the index before moving onto the next question.

In [120]:
%%sql 
drop index  g_batting_idx

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

#### Question 4ev:
Now, create a multiple column index on `g_batting` and `g_all` called `g_batting_g_all_idx` and record the query execution time and cost.

In [121]:
%%sql result_4v <<
CREATE INDEX g_batting_g_all_idx
ON appearances(g_batting, g_all);

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4v


In [122]:
!psql -h localhost -d baseball -c "explain analyze $result_4eiii_query_str"

                                                                                                                     QUERY PLAN                                                                                                                     
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=3608.08..3608.11 rows=1 width=21) (actual time=13.463..13.572 rows=216 loops=1)
   Group Key: salaries.playerid, salaries.yearid
   ->  Sort  (cost=3608.08..3608.09 rows=1 width=21) (actual time=13.450..13.466 rows=216 loops=1)
         Sort Key: salaries.playerid, salaries.yearid
         Sort Method: quicksort  Memory: 41kB
         ->  Hash Join  (cost=2871.30..3608.07 rows=1 width=21) (actual time=4.684..13.134 rows=216 loops=1)
               Hash Cond: ((salaries.yearid = appe

In [123]:
result_4ev_multiple_col_index_cost = 3613.82
result_4ev_multiple_col_index_timing = 14.702

In [124]:
grader.check("q4ev")

q4ev results: All test cases passed!

#### Question 4evi:
Given your findings from inspecting the query plans from `Question 4e`, answer the following question. Assign the variable `q4e_part6` to a list of all statements that are true.

1. Consider the following statements:
    1. Adding an index on a column used in an AND predicate will reduce the query time, but not the execution cost.
    1. Adding an index on a column used in an AND predicate will reduce the query cost, but not the execution time.
    1. Adding an index on a column used in an AND predicate will reduce the query cost and the execution time.
    1. Adding an index on a column used in an OR predicate will reduce the query time, but not the execution cost.
    1. Adding an index on a column used in an OR predicate will reduce the query cost, but not the execution time.
    1. Adding an index on a column used in an OR predicate will reduce the query cost and the execution time.
    1. Adding a mutlicolumn index on columns in an OR predicate will reduce the query time, but not the execution cost.
    1. Adding a mutlicolumn index on columns in an OR predicate will reduce the query cost, but not the execution time.
    1. Adding a mutlicolumn index on columns in an OR predicate will reduce the query cost and the execution time.

**Note:** Your answer should be formatted as follows: `q4e_part6 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q4evi
points: 2
-->

In [125]:
q4e_part6 = ['E','G']

In [126]:
grader.check("q4evi")

q4evi results: All test cases passed!

<!-- BEGIN QUESTION -->

#### Question 4evii

Explain your answer to the previous part based on your knowledge from lectures, and details of the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q4evii
manual: true
points: 2
-->

Adding an index can make the execution time either become really fast or slow. The index can make it become a pointers of where the data is stored in the tables which makes it faster to join. Indexes can also make the execution time slower since it has more indexes, the query would have to do more processing that would increase the cost. Adding indexes can make the costs go higher since it is creating more data and it makes the query go longer. Adding predicates can make the execution time faster since it is specify a certain type of data. Adding a predicate can cost the query more since it is processing the predicate but since it is picking a certain data and not looking through the whole table, the query time would be smaller. 

<!-- END QUESTION -->



## Question 5: More Indexes
### Question 5a:
Write 2 queries: one that finds the minimum salary from the salary table and one that finds the average. Inspect the queries' query plans and record their execution times and costs.

In [127]:
%sql select * from salaries limit 5

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
5 rows affected.


yearid,teamid,lgid,playerid,salary
1985,ATL,NL,barkele01,870000.0
1985,ATL,NL,bedrost01,550000.0
1985,ATL,NL,benedbr01,545000.0
1985,ATL,NL,campri01,633333.0
1985,ATL,NL,ceronri01,625000.0


In [128]:
%%sql result_5a_min << 
SELECT min(salaries.salary)
FROM salaries 


 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
1 rows affected.
Returning data to local variable result_5a_min


In [129]:
result_5a_min_query_str = "SELECT min(salaries.salary) FROM salaries "
!psql -h localhost -d baseball -c "explain analyze $result_5a_min_query_str"

                                                    QUERY PLAN                                                    
------------------------------------------------------------------------------------------------------------------
 Aggregate  (cost=525.35..525.36 rows=1 width=8) (actual time=4.586..4.587 rows=1 loops=1)
   ->  Seq Scan on salaries  (cost=0.00..459.28 rows=26428 width=8) (actual time=0.017..2.155 rows=26428 loops=1)
 Planning Time: 0.405 ms
 Execution Time: 4.662 ms
(4 rows)



In [130]:
result_5a_min_query_cost = 525.36
result_5a_min_query_timing = 4.650

In [131]:
grader.check("q5ai")

q5ai results: All test cases passed!

In [132]:
%%sql result_5a_avg <<
SELECT avg(salaries.salary) FROM salaries 

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
1 rows affected.
Returning data to local variable result_5a_avg


In [133]:
result_5a_avg_query_str = "SELECT avg(salaries.salary) FROM salaries "
!psql -h localhost -d baseball -c "explain analyze $result_5a_avg_query_str"

                                                    QUERY PLAN                                                    
------------------------------------------------------------------------------------------------------------------
 Aggregate  (cost=525.35..525.36 rows=1 width=8) (actual time=5.070..5.071 rows=1 loops=1)
   ->  Seq Scan on salaries  (cost=0.00..459.28 rows=26428 width=8) (actual time=0.019..2.134 rows=26428 loops=1)
 Planning Time: 0.376 ms
 Execution Time: 5.141 ms
(4 rows)



In [134]:
result_5a_avg_query_cost = 525.36
result_5a_avg_query_timing = 7.454

In [135]:
grader.check("q5aii")

q5aii results: All test cases passed!

### Question 5b:
Create an index on the `salary` column in the `salaries` table and re-inspect the query plans from the previous part and record the respective execution time and cost.

In [136]:
%%sql 
create index salary_idx on salaries(salary)

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

In [137]:
!psql -h localhost -d baseball -c "explain analyze $result_5a_min_query_str"

                                                                  QUERY PLAN                                                                  
----------------------------------------------------------------------------------------------------------------------------------------------
 Result  (cost=0.32..0.33 rows=1 width=8) (actual time=0.060..0.060 rows=1 loops=1)
   InitPlan 1 (returns $0)
     ->  Limit  (cost=0.29..0.32 rows=1 width=8) (actual time=0.057..0.058 rows=1 loops=1)
           ->  Index Only Scan using salary_idx on salaries  (cost=0.29..762.78 rows=26428 width=8) (actual time=0.056..0.056 rows=1 loops=1)
                 Index Cond: (salary IS NOT NULL)
                 Heap Fetches: 0
 Planning Time: 0.601 ms
 Execution Time: 0.111 ms
(8 rows)



In [138]:
result_5b_min_query_cost = 0.33
result_5b_min_query_timing = 0.158

In [139]:
grader.check("q5bi")

q5bi results: All test cases passed!

In [140]:
!psql -h localhost -d baseball -c "explain analyze $result_5a_avg_query_str"

                                                    QUERY PLAN                                                    
------------------------------------------------------------------------------------------------------------------
 Aggregate  (cost=525.35..525.36 rows=1 width=8) (actual time=7.268..7.270 rows=1 loops=1)
   ->  Seq Scan on salaries  (cost=0.00..459.28 rows=26428 width=8) (actual time=0.027..2.961 rows=26428 loops=1)
 Planning Time: 0.440 ms
 Execution Time: 7.347 ms
(4 rows)



In [141]:
result_5b_avg_query_cost = 525.36
result_5b_avg_query_timing = 5.851

In [142]:
grader.check("q5bii")

q5bii results: All test cases passed!

In the following questions, we will further explore the impact of indexes on query performance. To avoid any interference from the index on the `salary` column, drop the index before moving onto the next question.

In [143]:
%%sql 
drop index salary_idx

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

### Question 5c:
Given your findings from `Question 5a-b`, answer the following question. Assign the variable `q5c` to the correct answer choice.

1. Which of the following statements is true?
    1. An index on the column being aggregated in a query will always provide a performance enhancement.
    1. A query finding the MIN(salary) will always benefit from an index on salary, but a query finding MAX(salary) will not.
    1. A query finding the COUNT(salary) will always benefit from an index on salary, but a query finding AVG(salary) will not.
    1. Queries finding the MIN(salary) or MAX(salary) will always benefit from an index on salary, but queries finding AVG(salary) or COUNT(salary) will not.

*Note:* Your answer should be formatted as follows: `q5c = ['A', 'B']`

<!--
BEGIN QUESTION
name: q5c
points: 1
-->

In [144]:
q5c = ['C']

In [145]:
grader.check("q5c")

q5c results: All test cases passed!

<!-- BEGIN QUESTION -->

### Question 5d:

Explain your answer to the previous part based on your knowledge from lectures, and your inspection of the query plans.

<!--
BEGIN QUESTION
name: q5d
manual: true
points: 2
-->

Using count will benefit because it is just going through the whole table and counting rows rather than AVG where it going through the whole table and using a aggregating through the whole table. The AVG function is taking longer to process and will cost more.  

<!-- END QUESTION -->



## Question 6: Clustered Indexes
In this question, we will inspect the impact that clustering our data on an index can have on a query's performance.

### Question 6a
Write a query that finds the `playerid`, `yearid`, `teamid`, and `ab` for all players whose `ab` was above 500. Inspect the query plan and record the execution time and cost.

In [146]:
%sql select * from batting limit 5

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
5 rows affected.


playerid,yearid,stint,teamid,lgid,g,ab,r,h,h2b,h3b,hr,rbi,sb,cs,bb,so,ibb,hbp,sh,sf,gidp
abercda01,1871,1,TRO,NA,1,4,0,0,0,0,0,0,0,0,0,0,None,None,None,None,0
addybo01,1871,1,RC1,NA,25,118,30,32,6,0,0,13,8,1,4,0,None,None,None,None,0
allisar01,1871,1,CL1,NA,29,137,28,40,4,5,0,19,3,1,2,5,None,None,None,None,1
allisdo01,1871,1,WS3,NA,27,133,28,44,10,2,2,27,1,1,0,2,None,None,None,None,0
ansonca01,1871,1,RC1,NA,25,120,29,39,11,3,0,16,6,2,2,1,None,None,None,None,0


In [147]:
%%sql result_6a <<
select playerid, yearid, teamid, ab from batting where ab > 500;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
8839 rows affected.
Returning data to local variable result_6a


In [148]:
# Do not delete/edit this cell
result_6a.DataFrame().sort_values(['playerid', 'yearid', 'teamid']).iloc[:1000].to_csv('results/result_6a.csv', index=False)

In [149]:
grader.check("q6ai")

q6ai results: All test cases passed!

In [150]:
result_6a_query_str = "select playerid, yearid, teamid, ab from batting where ab > 500;"
!psql -h localhost -d baseball -c "explain analyze $result_6a_query_str"

                                                 QUERY PLAN                                                 
------------------------------------------------------------------------------------------------------------
 Seq Scan on batting  (cost=0.00..2884.05 rows=8970 width=21) (actual time=0.267..13.485 rows=8839 loops=1)
   Filter: (ab > 500)
   Rows Removed by Filter: 95485
 Planning Time: 0.337 ms
 Execution Time: 13.883 ms
(5 rows)



In [151]:
result_6a_cost = 2884.05
result_6a_timing = 13.737

In [152]:
grader.check("q6aii")

q6aii results: All test cases passed!

### Question 6b:
Cluster the `batting` table on its primary key (hint: use `\di` to find out what name of the primary key is). We are able to directly cluster on the primary key (without first creating a separate index) because Postgres automatically creates an index for it. Re-inspect the query plan for the query from `Question 6a` and record the execution time and cost.

In [153]:
%%sql 
CLUSTER batting USING batting_pkey

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

In [154]:
!psql -h localhost -d baseball -c "explain analyze $result_6a_query_str"

                                                 QUERY PLAN                                                 
------------------------------------------------------------------------------------------------------------
 Seq Scan on batting  (cost=0.00..2878.05 rows=8970 width=21) (actual time=0.024..15.766 rows=8839 loops=1)
   Filter: (ab > 500)
   Rows Removed by Filter: 95485
 Planning Time: 0.535 ms
 Execution Time: 16.213 ms
(5 rows)



In [155]:
result_6b_cost = 2878.05
result_6b_timing = 22.283

In [156]:
grader.check("q6b")

q6b results: All test cases passed!

### Question 6c
Now, let's try clustering the table based on another index. Create an index on `ab` column called `ab_idx` in the `batting` table AND cluster the `batting` table with this new index. Now, re-inspect the query plan and record the execution time and cost.

In [157]:
%%sql 
create index ab_idx on batting(ab)
CLUSTER batting USING ab_idx


 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
(psycopg2.errors.SyntaxError) syntax error at or near "CLUSTER"
LINE 2: CLUSTER batting USING ab_idx
        ^

[SQL: create index ab_idx on batting(ab)
CLUSTER batting USING ab_idx]
(Background on this error at: https://sqlalche.me/e/14/f405)


In [158]:
!psql -h localhost -d baseball -c "explain analyze $result_6a_query_str"

                                                 QUERY PLAN                                                 
------------------------------------------------------------------------------------------------------------
 Seq Scan on batting  (cost=0.00..2878.05 rows=8970 width=21) (actual time=0.022..15.183 rows=8839 loops=1)
   Filter: (ab > 500)
   Rows Removed by Filter: 95485
 Planning Time: 0.354 ms
 Execution Time: 15.554 ms
(5 rows)



In [159]:
result_6c_cost = 1788.37
result_6c_timing = 2.386

In [160]:
grader.check("q6c")

q6c results: All test cases passed!

### Question 6d
Given your findings from inspecting the query plans from `Question 6a-c`, answer the following question. Assign the variable `q6d` to a list of all statements that are true.

1. Consider the following statements:
    1. Clustering based on the `ab_idx` decreased the cost of the query.
    1. Clustering based on the `ab_idx` increased the cost of the query.
    1. Clustering based on the `ab_idx` increased the execution time of the query.
    1. Clustering based on the `ab_idx` decreased the execution time of the query.
    1. Clustering based on the `batting_pkey` decreased the cost of the query.
    1. Clustering based on the `batting_pkey` increased the cost of the query.
    1. Clustering based on the `batting_pkey` increased the execution time of the query.
    1. Clustering based on the `batting_pkey` decreased the execution time of the query.
    1. None of the above
    
**Note:** Your answer should be formatted as follows: `q6d = ['A', 'B']`

<!--
BEGIN QUESTION
name: q6d
points: 2
-->

In [161]:
q6d = ['A', 'D', 'F', 'H' ]

In [162]:
grader.check("q6d")

q6d results: All test cases passed!

<!-- BEGIN QUESTION -->

### Question 6e:

Explain your answer to the previous part based on your knowledge from lectures, and your inspection of the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q6e
manual: true
points: 2
-->

Clustering data makes the cost of the query go down because it reduces the amount of data processed by queries. The cost goes down because there isn't much data being processed. Since there is less data being processed, the execution time is shorter since there is a smaller data set to process. 

<!-- END QUESTION -->



## Question 7: Cost of Index Management
Until now, we have seen the positive potential impact that indexes can have on query performance, but remember in real world technologies/applications, we will be routinely receiving new data (and in large quantities) which would trigger regular updates to our tables. In this section, we will dive into the cost of managing the indexes that we create.

### Question 7a:
Record the time it takes to insert 300,000 rows into the `salaries` table when no additional index is configured.

Run the following cell to setup a column to track which rows we added as part of these inserts.

In [163]:
%sql ALTER TABLE salaries ADD added boolean DEFAULT False;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

Next, run the provided update script and record the **wall time**.

**NOTE:** Running the below cell multiple times may result in an error, unless you first delete the rows with the cell given at the end of this subpart.

In [164]:
%%time
%%sql
DO $$
 DECLARE counter INTEGER := 1;
 BEGIN
     FOR counter IN 100001..400000 LOOP
     INSERT INTO salaries (yearid, teamid, lgid, playerid, salary, added)
         VALUES (2021, 'ATL', 'NL', 'p' || counter, RANDOM() * 1000000, true);
     END LOOP;
END;
$$;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
CPU times: user 3.94 ms, sys: 1.99 ms, total: 5.92 ms
Wall time: 3.86 s


[]

In [165]:
result_7a_timing = 3.81

In [166]:
grader.check("q7a")

q7a results: All test cases passed!

Before adding an index to the salaries table and re-timing the updates, delete all the rows that were added to the table from the update script.

In [167]:
%%sql
DELETE FROM salaries
WHERE added = 'true';

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
300000 rows affected.


[]

### Question 7b:
Now, create an index on the `salary` column and record the **wall time** after executing the update script.

In [168]:
%%sql 
create index salary_idx on salaries(salary)

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

**NOTE:** Running the below cell multiple times may result in an error, unless you first delete the rows with the cell given at the end of last subpart.

In [169]:
%%time
%%sql
DO $$
 DECLARE counter INTEGER := 1;
 BEGIN
     FOR counter IN 100001..400000 LOOP
     INSERT INTO salaries (yearid, teamid, lgid, playerid, salary, added)
         VALUES (2021, 'ATL', 'NL', 'p' || counter, RANDOM() * 1000000, true);
     END LOOP;
END;
$$;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
CPU times: user 3.21 ms, sys: 4.05 ms, total: 7.26 ms
Wall time: 6.14 s


[]

In [170]:
result_7b_timing = 6.91

In [171]:
grader.check("q7b")

q7b results: All test cases passed!

<!-- BEGIN QUESTION -->

### Question 7c:
What difference did you notice when you added an index into the salaries table and re-timed the update? Why do you think it happened?

<!--
BEGIN QUESTION
name: q7c
manual: true
points: 2
-->

I noticed that the time doubled by creating an index. I think the time doubled because since an index was added it took longer for script to excute since there was more data. 

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

## Question 8: Takeaway

In this project, we explored how the database system optimizes query execution and how users can futher tune the performance of their queries.

Familiarizing yourself with these optimization and tuning methods will make you a better data engineer. In this question, we'll ask you to recall and summarize these concepts. Who knows? Maybe one day it will help you during an interview or on a project.

In the following answer cell,
1. Name 3 methods you learned in this project. The method can be either the optimization done by the database system, or the fine tuning done by the user.
2. For each method, summarize how and why it can optimize query performance. Feel free to discuss any drawbacks, if applicable.

<!--
BEGIN QUESTION
name: q8
manual: true
points: 6
-->

I learned about when tables will use hash joins, nested loop joins, and merge sorts. Tables will use hash joins if you are naturally joining two tables, merge sorts if the two tables are already being sorted, and a nested loop join if you are working on a small amount of data. I also learned that adding indexes can increase the cost and executation time greatly because you making the query process through more data. I have also learned that if you want to reduce costs and the query time, using clustering is the best way to decrease it. 

<!-- END QUESTION -->



## Congratulations! You have finished Project 2.

Run the following cell to zip and download the results of your queries. You will also need to run the export cell at the end of the notebook.

**Please save your notebook before exporting (this is a good time to do it!)** Otherwise, we may not be able to register your written responses.

**For submission on Gradescope, you will need to submit BOTH the proj2.zip file genreated by the export cell and the results.zip file generated by the following cell.**

**Common submission issues:** You MUST submit the generated zip files (not folders) to the autograder. However, Safari is known to automatically unzip files upon downloading. You can fix this by going into Safari preferences, and deselect the box with the text "Open safe files after downloading" under the "General" tab. If you experience issues with downloading via clicking on the link, you can also navigate to the project 2 directory within JupyterHub (remove `proj2.ipynb` from the url), and manually download the generated zip files. Please post on Ed if you encounter any other submission issues.

In [172]:
from IPython.display import display, FileLink

!zip -r results.zip results
results_file = FileLink('./results.zip', result_html_prefix="Click here to download: ")
display(results_file)

updating: results/ (stored 0%)
updating: results/result_1a.csv (deflated 53%)
updating: results/result_1c.csv (deflated 64%)
updating: results/result_1di_view.csv (deflated 37%)
updating: results/result_1di_no_view.csv (deflated 37%)
updating: results/result_1ei.csv (deflated 64%)
updating: results/result_1eii.csv (deflated 37%)
updating: results/result_4a.csv (deflated 59%)
updating: results/result_4ei.csv (deflated 62%)
updating: results/result_4eiii.csv (deflated 64%)
updating: results/result_6a.csv (deflated 72%)


/home/jovyan/fa22/proj/proj2/results.zip

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [173]:
grader.check_all()

q0 results: All test cases passed!

q1a results: All test cases passed!

q1bi results: All test cases passed!

q1bii results: All test cases passed!

q1c results: All test cases passed!

q1di results: All test cases passed!

q1dii results: All test cases passed!

q1ei results: All test cases passed!

q1eii results: All test cases passed!

q1eiii results: All test cases passed!

q1eiv results: All test cases passed!

q2a results: All test cases passed!

q2b results: All test cases passed!

q2c results: All test cases passed!

q3a results: All test cases passed!

q3b results: All test cases passed!

q3c results: All test cases passed!

q4ai results: All test cases passed!

q4aii results: All test cases passed!

q4b results: All test cases passed!

q4c results: All test cases passed!

q4di results: All test cases passed!

q4ei_part1 results: All test cases passed!

q4ei_part2 results: All test cases passed!

q4eii results: All test cases passed!

q4eiii_part1 results: All test cases passe

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [174]:
# Save your notebook first, then run this cell to export your submission.
grader.export()